In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Chargement des données

In [36]:
# Chargement données
df = pd.read_csv('raw/retail_sales_dirty.csv')

# Exploration du Dataframe

## Informations Basiques

Format du dataframe, Colonnes, Types de Colonnes, Aperçu, Info

In [37]:
# Exploration du dataframe
print(f"Shape: {df.shape}")
print(f"\nColonnes:\n{df.columns.tolist()}")
print(f"\nTypes:\n{df.dtypes}")
print("\nInfo")
df.info()

Shape: (12575, 11)

Colonnes:
['Transaction ID', 'Customer ID', 'Category', 'Item', 'Price Per Unit', 'Quantity', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date', 'Discount Applied']

Types:
Transaction ID       object
Customer ID          object
Category             object
Item                 object
Price Per Unit      float64
Quantity            float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
Discount Applied     object
dtype: object

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          1197

In [38]:
print("\nAperçu:")
df.head(10)


Aperçu:


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
5,TXN_7482416,CUST_09,Patisserie,NaN,NaN,10.0,200.0,Credit Card,Online,2023-11-30,NaN
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,2023-06-10,True
7,TXN_1372952,CUST_21,Furniture,NaN,33.5,NaN,NaN,Digital Wallet,In-store,2024-04-02,True
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-store,2023-04-26,False
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3.0,109.5,Cash,Online,2024-03-14,False


### Temporalité

In [39]:
print(f"Date première transaction : {df['Transaction Date'].min()}")
print(f"Date dernière transaction : {df['Transaction Date'].max()}")

Date première transaction : 2022-01-01
Date dernière transaction : 2025-01-18


### Détails des Données Manquantes

In [40]:
df_Nul = pd.DataFrame({
    'Count': df.isnull().sum(),
    'Pourcentage': (df.isnull().sum() / df.shape[0] * 100).round(2)
    })
df_Nul=df_Nul[df_Nul["Count"]>0]
df_Nul

,Count,Pourcentage
Item,1213,9.65
Price Per Unit,609,4.84
Quantity,604,4.80
Total Spent,604,4.80
Discount Applied,4199,33.39


### Doublons

In [41]:

# Doublons
print(f"Doublons d'entrées : {df.duplicated().sum()}")
print(f"\nUnicité des Transaction ID : {df['Transaction ID'].duplicated().sum()}")



Doublons d'entrées : 0

Unicité des Transaction ID : 0


Pas de doublons identifiés

### Anomalies et Incohérences

- Vérification que Price Per Unit et Total Spent ne soient pas négatif
- Vérification que Quantity soit ni nul et ni négatif
- Vérification de la cohérence Price Per Unit x Quantity = Total Spent


In [42]:
print(f"\nPrix négatifs : {(df['Price Per Unit'] < 0).sum()}")
print(f"Quantités nulles ou négatives : {(df['Quantity'] <= 0).sum()}")
print(f"Totaux négatifs : {(df['Total Spent'] < 0).sum()}")


Prix négatifs : 0
Quantités nulles ou négatives : 0
Totaux négatifs : 0


In [43]:
df_calc=df.copy()
df_calc['Calculated_Total'] = df_calc['Price Per Unit'] * df_calc['Quantity']
df_calc['Diff'] = (df_calc['Total Spent'] - df_calc['Calculated_Total']).abs()

Incoherence = df_calc[df_calc['Diff'] > 0.01].shape[0]

print(f"{Incoherence} Incohérence(s) détectée(s)")


0 Incohérence(s) détectée(s)


In [44]:
#Statistique sur les colonnes number du dataframe
df.select_dtypes(include='number').describe()

,Price Per Unit,Quantity,Total Spent
count,11966.000000,11971.000000,11971.000000
mean,23.365912,5.536380,129.652577
std,10.743519,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


### Valeurs des colonnes catégorielles

In [45]:
df['Category'].unique()

array(['Patisserie', 'Milk Products', 'Butchers', 'Beverages', 'Food',
       'Furniture', 'Electric household essentials',
       'Computers and electric accessories'], dtype=object)

In [46]:
df['Payment Method'].unique()

array(['Digital Wallet', 'Credit Card', 'Cash'], dtype=object)

In [47]:
df['Location'].unique()

array(['Online', 'In-store'], dtype=object)

In [48]:
df['Discount Applied'].unique()

array([True, False, nan], dtype=object)

### Bilan

**Structure :**
- 11 colonnes, 12 575 lignes
- Période : du 2022-01-01 au 2025-01-18

**Types incorrects :**
- Transaction Date  à convertir en datetime
- Discount Applied  à convertir en boolean

**Valeurs manquantes :**
- Item : 9,65% (1213 lignes)
- Price Per Unit : 4,84% (609 lignes)
- Quantity : 4,80% (604 lignes)
- Total Spent : 4,80% (604 lignes)
- Discount Applied : 33,39% (4199 lignes)

**Qualité des données :**
- Pas de doublons
- Pas d'incohérence arithmétique 
- Pas de valeurs négatives
- Valeurs catégorielles cohérentes 

**Plan de nettoyage :**
1. Convertir Transaction Date en datetime
2. Convertir Discount Applied en boolean
3. Traiter les valeurs manquantes (Correction à définir)
4. Exporter le dataset nettoyé

# Nettoyage des données

## Conversion de type

In [49]:
# Conversion de type

# Transaction Date
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])


# Discount Applied
df['Discount Applied'] = df['Discount Applied'].astype('boolean')

# Contrôle
print("\nContrôle conversion\n")
print(df.dtypes)


Contrôle conversion

Transaction ID              object
Customer ID                 object
Category                    object
Item                        object
Price Per Unit             float64
Quantity                   float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
Discount Applied           boolean
dtype: object


## Remplacement des valeurs nulles

In [50]:


# Calcul du prix unitaire quand total et quantité connus
mask_price = (df['Price Per Unit'].isna()) & (df['Total Spent'].notna()) & (df['Quantity'].notna()) & (df['Quantity'] > 0)
df.loc[mask_price, 'Price Per Unit'] = df.loc[mask_price, 'Total Spent'] / df.loc[mask_price, 'Quantity']


# Calcul de la quantité quand total et prix unitaire connus
mask_qty = (df['Quantity'].isna()) & (df['Total Spent'].notna()) & (df['Price Per Unit'].notna()) & (df['Price Per Unit'] > 0)
df.loc[mask_qty, 'Quantity'] = df.loc[mask_qty, 'Total Spent'] / df.loc[mask_qty, 'Price Per Unit']


# Calcul total quand prix et quantité connus
mask_total = (df['Total Spent'].isna()) & (df['Price Per Unit'].notna()) & (df['Quantity'].notna())
df.loc[mask_total, 'Total Spent'] = df.loc[mask_total, 'Price Per Unit'] * df.loc[mask_total, 'Quantity']




In [51]:
# Colonne Discount Applied remplacement NaN par False
df['Discount Applied'] = df['Discount Applied'].fillna(False)
print(f"✓ Discount Applied : NaN remplacés par False")

print("\n=== ÉTAT APRÈS RECALCULS ===")
print(f"\nValeurs manquantes restantes :")
print(df.isnull().sum()[df.isnull().sum() > 0])

✓ Discount Applied : NaN remplacés par False

=== ÉTAT APRÈS RECALCULS ===

Valeurs manquantes restantes :
Item           1213
Quantity        604
Total Spent     604
dtype: int64


In [52]:
# Suppression des lignes non exploitables (Quantity OU Total Spent manquants)

df_clean = df[(df['Quantity'].notna()) & (df['Total Spent'].notna())].copy()

print(f"\nBilan")
print(f"Lignes conservées : {len(df_clean)} / {len(df)}")
print(f"Lignes supprimées : {len(df) - len(df_clean)} ({((len(df) - len(df_clean)) / len(df) * 100):.1f}%)")

print(f"\n✓ Valeurs manquantes restantes :")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])


Bilan
Lignes conservées : 11971 / 12575
Lignes supprimées : 604 (4.8%)

✓ Valeurs manquantes restantes :
Item    609
dtype: int64


In [53]:
# Export
fout='retail_sales_clean.csv'
df_clean.to_csv(fout, index=False)
print(f"\n Dataset nettoyé exporté : {fout}")



 Dataset nettoyé exporté : retail_sales_clean.csv
